量子ウォークとランダムウォークをプロットするソースコード  
時間発展演算子 $U(t) = e^{i t \frac{A}{2}}$  (あるいは $U(t) = e^{i t \frac{A}{2}}$) 、$A$ を隣接行列として計算した場合

In [ ]:
# 完成版

import numpy as np
from scipy.linalg import expm
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Output

波動関数 $\psi(t)$ を x1×1の行列とする。これは、1次元軸上に粒子が x1 個並んでいる状態を表している。
また、$$\psi(t) = U(t) \psi(0)$$($\psi(0) = initialstate$) である。

In [ ]:
# 有効数字を3桁に設定
#np.set_printoptions(precision=1)

# グラフのサイズ
x1 = 101

# 初期状態ベクトルを定義
initial_state = np.zeros((x1, 1))
initial_state[50]= 1
#initial_state[140]= 0.4
#initial_state[100]= 0.1
#print(initial_state)

# 出力エリアの作成
output = Output()

def quantum_walk(c1, c2, t1):
    with output:
        output.clear_output(wait=True)  # 出力をクリア
        # 隣接行列の作成
        A = np.zeros((x1, x1))
        
        for i in range(x1):
            A[i, (i + 1) % x1] = c1
            A[i, (i - 1) % x1] = c1
            A[i, i%x1] = 1 - c1

            #追加の補足
            A[x1-1,0] = c1
            A[0,x1-1] = c1

            #A[0,1] = 2*c1
            #A[2,1] = 2*c1

        #c2は二次元格子用　現在は使用しない
            
        #print(A)
        #print("Aはユニタリ？{}".format(np.dot(A,A.T) ))

        # 時間発展演算子 U_t の計算
        U_t = expm(-1j * t1 * A / 2)
        #print("-1j*t1*A/2={}".format(-1j * t1 * A / 2))
        #print(U_t)

        absU = 0
        for p1 in range(x1):
            absU += abs(U_t[p1,int((x1-1)/2)])**2
        #print(absU)
        #print((A)**2)
        #print("U_tはユニタリ?  {}".format(np.dot(U_t,U_t.T)))
        # 時間発展後の状態を計算
        final_state = U_t @ initial_state
        #final_state = U_t @ np.array(initial_state).T

        if t1 % 1.0 == 0:
            A_yobi = np.copy(A)
            for i in range(int(t1)):
                A_yobi = np.dot(A_yobi, A)
            R_t = np.copy(A_yobi)

        final_state2 = R_t @ initial_state
        probabilities2 = np.abs(final_state2)**2
        print("ランダムウォークの存在確率の合計は1になるか{}".format(np.sum(final_state2)))
        # 状態の確率分布をプロット
        probabilities = np.abs(final_state)**2
        #print(probabilities)
        #probabilities = np.abs(final_state)
        plt.figure(figsize=(10, 2))
        plt.bar(range(x1), probabilities.flatten())
        plt.xlabel('Node')
        plt.ylabel('Probability')
        plt.title(f'Probability Distribution after time t=1 (c1={c1}, c2={c2})')
        plt.show()

        plt.figure(figsize=(10, 2))
        plt.bar(range(x1), probabilities2.flatten())
        plt.xlabel('Node')
        plt.ylabel('Probability')
        plt.title(f'Probability Distribution after time2 t=1 (c1={c1}, c2={c2})')
        plt.show()

In [ ]:
# スライダーバーの定義
c1_slider = FloatSlider(min=0, max=20, step=0.1, value=1, description='c1')
c2_slider = FloatSlider(min=0, max=20, step=0.1, value=1, description='c2')
t1_slider = FloatSlider(min=0, max=500, step=1.0, value=1, description='t1')

# インタラクティブ機能の起動
interact(quantum_walk, c1=c1_slider, c2=c2_slider, t1=t1_slider);

# スライダーの下に出力を配置
display(output)